Let $(\rho, V )$ be an irreducible representation of $G$ with character $\chi$. An argument using Schur's lemma shows that $\rho(b_i) = \sum_{g \in C_i}\rho(g)$ is a scalar matrix $\lambda_i I$ since $\rho(b_i)$ commutes with all representation matrices. Indeed,
\begin{equation}
    \rho(h)\rho(b_i)\rho(h)^{-1} = \sum_{g \in C_i} \rho(h)\rho(g)\rho(h^{-1}) = \sum_{g \in C_i} \rho(hgh^{-1})
\end{equation}
As $g$ runs through all elements of the conjugacy class $C_i$, the element $hgh^{-1}$ also runs through all elements of $C_i$. Therefore, the sum remains unchanged $\sum_{g' \in C_i} \rho(g') = \rho(b_i)$. We have shown $\rho(h)\rho(b_i)\rho(h)^{-1} = \rho(b_i)$, which means $\rho(h)\rho(b_i) = \rho(b_i)\rho(h)$, as required.

We wish to determine the scalar $\lambda$. Taking the trace,
\begin{equation}
    \mathop{tr}(\rho(b_i)) = \mathop{tr}\left(\sum_{g \in C_i} \rho(g)\right) = \sum_{g \in C_i} \mathop{tr}(\rho(g)) = \chi(g).
\end{equation}
Since characters are constant on conjugacy classes, $\chi(g) = \chi(g_i)$ for any $g \in C_i$, where $g_i$ is a representative of the class $C_i$, we have
\begin{equation}
    \sum_{g \in C_i} \chi(g_i) = |C_i|\chi(g_i)
\end{equation}
Let $d = \dim(V)$ be the dimension of the representation. We know $d = \chi(e)$, so
\begin{equation}
    \mathop{tr}(\lambda_iI) = d\lambda_i = \chi(e)\lambda_i.
\end{equation}
We conclude that $|C_i|\chi(g_i) = \chi(e) \lambda_i$ which solves for
\begin{equation}
    \lambda_i = \frac{|C_i|\chi(g_i)}{\chi(e)}.
\end{equation}

In [1]:
import numpy as np
import collections
import random
import re

class Permutation:
    '''
    Represents a permutation on the set {0, 1, ..., n-1}.
    '''
    def __init__(self, mapping):
        self.mapping = tuple(mapping)
        self.size = len(mapping)

    def __mul__(self, other):
        '''
        Composition of permutations (self after other).
        '''
        return Permutation([self.mapping[other.mapping[i]] for i in range(self.size)])

    def inverse(self):
        '''
        Computes the inverse of the permutation.
        '''
        inv_mapping = [0] * self.size
        for i in range(self.size):
            inv_mapping[self.mapping[i]] = i
        return Permutation(inv_mapping)

    def __eq__(self, other):
        return self.mapping == other.mapping

    def __hash__(self):
        return hash(self.mapping)

    def __repr__(self):
        # Convert to 1-based cycle notation for readability
        p = list(self.mapping)
        n = len(p)
        cycles = []
        seen = [False] * n
        for i in range(n):
            if not seen[i]:
                cycle = []
                j = i
                while not seen[j]:
                    seen[j] = True
                    cycle.append(j + 1)
                    j = p[j]
                if len(cycle) > 1:
                    cycles.append(tuple(cycle))
        return str(cycles) if cycles else "()"

    def __lt__(self, other):
        return self.mapping < other.mapping

def parse_cycle_notation(cycle_str, n):
    '''
    Parses a string in cycle notation into a Permutation object.
    '''
    mapping = list(range(n))
    cycle_str = ''.join(cycle_str.split())
    if not cycle_str or cycle_str == "()":
        return Permutation(mapping)

    cycles_raw = cycle_str.replace(")(", "|").replace("(", "").replace(")", "")
    for cycle_s in cycles_raw.split("|"):
        try:
            cycle = [int(x) for x in cycle_s.split(',')]
            if len(cycle) > 1:
                # 0-based cycle
                cycle_0based = [x - 1 for x in cycle]
                last_val = mapping[cycle_0based[-1]]
                for i in range(len(cycle_0based) - 1, 0, -1):
                    mapping[cycle_0based[i]] = mapping[cycle_0based[i-1]]
                mapping[cycle_0based[0]] = last_val
        except (ValueError, IndexError):
            continue
    return Permutation(mapping)

def generate_group(generators):
    '''
    Generates all elements of a group from a set of generators.
    '''
    if not generators: return set()
    identity = Permutation(tuple(range(generators[0].size)))
    group = {identity}
    worklist = collections.deque([identity])
    while worklist:
        current = worklist.popleft()
        for gen in generators:
            new = current * gen
            if new not in group:
                group.add(new)
                worklist.append(new)
    return group

def find_conjugacy_classes(group):
    '''
    Finds the conjugacy classes of a group.
    '''
    if not group: return []
    classes = []
    processed = set()
    # Sort elements to get deterministic representatives
    for g in sorted(list(group)):
        if g not in processed:
            current_class = {h * g * h.inverse() for h in group}
            classes.append(sorted(list(current_class)))
            processed.update(current_class)
    # Sort classes by size, then by representative for canonical order
    classes.sort(key=lambda c: (len(c), c[0]))
    return [set(c) for c in classes]

def compute_nu_ijk(classes, group_order):
    '''
    Computes the structure constants v_ijk.
    '''
    r = len(classes)
    nu = np.zeros((r, r, r), dtype=int)

    element_to_class_idx = {g: k for k, class_k in enumerate(classes) for g in class_k}

    for i in range(r):
        for j in range(r):
            # Pick representatives to compute the product b_i * b_j
            product_counts = collections.defaultdict(int)
            for g1 in classes[i]:
                for g2 in classes[j]:
                    product = g1 * g2
                    product_counts[element_to_class_idx[product]] += 1

            for k, count in product_counts.items():
                # v_ijk = count of times an element of C_k appears in C_i*C_j
                # The coefficient of each element in a class sum b_k is the same.
                nu[i, j, k] = count / len(classes[k])
    return nu

We have the relation $b_ib_j = \sum_k \nu_{ijk}b_k$. Since $\rho$ is a group algebra homomorphism, we obtain
\begin{equation}
    \rho(b_ib_j) = \rho(b_i)\rho(b_j) = \sum_k \nu_{ijk}\rho(b_k).
\end{equation}
We replace each $\rho(b_k)$ with $λ_kI$,
\begin{equation}
\lambda_i\lambda_j = (λ_iI)(λ_jI) = \sum_k \nu_{ijk} (\lambda_k I) = \left(\sum_k \nu_{ijk}\lambda_k\right) I.
\end{equation}
By equating the scalar coefficients of the identity matrix, we obtain $\lambda_i\lambda_j = \sum_k \nu_{ijk}\lambda_k$. The right-hand coefficient can be recognised as the $j$-th component of a matrix-vector product.

Recall that $N_i$ is the matrix with $(j, k)$-th entry $\nu_{ijk}$. The $j$-th component of the product $N_i\lambda$ is
\begin{equation}
    (N_i w)_j = \sum_k (N_i)_{jk} \lambda_k = \sum_k \nu_{ijk} \lambda_k = \sum_k \nu_{ijk}\lambda_k.
\end{equation}
Therefore, the equation becomes $\lambda_i \lambda_j = (N_i \lambda)_j$ and since this holds for all components $j = 1, \dots, r$, we have the matrix equation
\begin{equation}
    N_i \lambda = \lambda_i \lambda.
\end{equation}
This shows precisely that the matrix $N_i$ has an eigenvalue $\lambda_i$ with eigenvector components $\lambda_j = |C_j|\chi(g_j) / \chi(e)$. Any non-zero scalar multiple of this is also an eigenvector for the same eigenvalue, thus the vector
\begin{equation}
    v = \begin{pmatrix}
        |C_1|\chi(g_1) \\
        \vdots \\
        |C_r|\chi(g_r)
    \end{pmatrix}
\end{equation}
where $g_i \in C_i$ is an eigenvector for each of the matrices $N_1, \dots, N_r$ with eigenvalues given by $\lambda_i = |C_i|\chi(g_i) / \chi(e)$.


---

For each of the $r$ distinct irreducible characters $\chi^{(1)}, \dots, \chi^{(r)}$ of $G$, we have constructed a common eigenvector $v^{(\alpha)}$ for the set of matrices $\{N_1, ..., N_r\}$. These $r$ eigenvectors are linearly independent because the characters themselves are linearly independent. Take a linear combination
\begin{equation}
    M = c_1N_1 + c_2N_2 + \cdots + c_rN_r.
\end{equation}
Since $v^{(\alpha)}$ is a common eigenvector of all N_i, it is also an eigenvector of M
\begin{equation}
    Mv^{(\alpha)} = \sum_i c_iN_iv^{(\alpha)} = \sum_i c_i\lambda_i^{(\alpha)}v^{(\alpha)}
\end{equation}
where $\lambda_i^{(\alpha)} = |C_i|\chi^{(\alpha)}(g_i)/\chi^{(\alpha)}(e)$ is the eigenvalue of $N_i$ for eigenvector $v^{(\alpha)}$. The eigenvalue of $M$ corresponding to $v^{(\alpha)}$ is $\mu^{(\alpha)} = \sum_i c_i\lambda_i^{(\alpha)}$. We want to choose coefficients $c = (c_1, \dots, c_r)$ such that all $r$ eigenvalues $\mu^{(1)}, \dots, \mu^{(r)}$ are distinct. The condition $\mu^{(\alpha)} = \mu^{(\beta)}$ is equivalent to
\begin{equation}
    \sum_i c_i (\lambda_i^{(\alpha)} - \lambda_i^{(\beta)}) = 0.
\end{equation}
For any fixed pair $\alpha \neq \beta$, since the irreducible characters $\chi^{(\alpha)}$ and $\chi^{(\beta)}$ are different, the corresponding rows in the character table are different. There must be at least one $i$ for which $\lambda_i^{(\alpha)} \neq \lambda_i^{(\beta)}$. Therefore, the vector $d^{(\alpha,\beta)}$ with components $d_i = \lambda_i^{(\alpha)} - \lambda_i^{(\beta)}$ is a non-zero vector. The equation $\sum_i c_id_i = 0$ defines a hyperplace in $\mathbb{C}^r$. There are $r(r-1)/2$ pairs of distinct indices $(\alpha, \beta)$ and finitely many hyperplanes cannot cover the entirity of $\mathbb{C}^r$, so we can always find coefficients $c_i$ such that $\mu^{(\alpha)}$ are distinct, hence $M$ has $r$ distinct eigenvalues. In fact, the linear combination
\begin{equation}
    M(k) = N_1 + k N_2 + k^2 N_3 + \cdots + k^{r-1} N_r
\end{equation}
is guaranteed to have $r$ distinct eigenvalues for some $k$. The eigenvalue of this matrix for the eigenvector $v^{(\alpha)}$ is
\begin{equation}
    \mu^{(\alpha)}(k) = \lambda_1^{(\alpha)} + k \lambda_2^{(\alpha)} + k^2 \lambda_3^{(\alpha)} + \cdots + k^{r-1} \lambda_r^{(\alpha)},
\end{equation}
which is a polynomial in $k$ of degree at most $r-1$. Suppose for a specific value of $k$, that two eigenvalues are the same, i.e., $\mu^{(\alpha)}(k) = \mu^{(\beta)}(k)$ for two different characters $a \neq b$. This would mean that
\begin{equation}
    (\lambda_1^{(\alpha)} - \lambda_1^{(\beta)}) + k(\lambda_2^{(\alpha)} - \lambda_2^{(\beta)}) + \cdots + k^{r-1}(\lambda_r^{(\alpha)} - \lambda_r^{(\beta)}) = 0.
\end{equation}
Since the irreducible characters are distinct, their corresponding vectors of eigenvalues must also be distinct. This guarantees that at least one of the differences $(\lambda_i^{(\alpha)} - \lambda_i^{(\beta)})$ is non-zero, so the above polynomial is of degree at most $r-1$, thus has at most $r-1$ roots. This means that for any pair of distinct characters $(\alpha, \beta)$ of which there are only finitely many, there are at most $r-1$ integer values of $k$ that could cause their eigenvalues to clash.

The inner product of characters $\chi_1$ and $\chi_2$ is \begin{equation}
    \langle \chi_1, \chi_2 \rangle = \frac{1}{|G|} \sum_{g \in G} \chi_1(g)\overline{\chi_2(g)}.
\end{equation}
Computing the simultaneous eigenvectors of $N_1, \dots, N_r$ determines each row of the character table up to a scalar multiple. The scaling of each row is uniquely determined by the requirement that for each irreducible character $\chi$ we have $\langle \chi, \chi \rangle = 1$ and $\chi(1) > 0$.

In [2]:
def compute_character_table(generators):
    '''
    Computes the character table for a group given by generators.
    '''
    # Generate group and find canonically sorted classes
    group = generate_group(generators)
    order = len(group)
    classes = find_conjugacy_classes(group)
    r = len(classes)
    class_sizes = [len(c) for c in classes]

    # Compute structure constants and matrices N_i
    nu = compute_nu_ijk(classes, order)
    matrices_N = [nu[i, :, :] for i in range(r)]

   # Deterministic search for a suitable linear combination
    for k in range(r * r):
        # Construct M(k) = N1 + k*N2 + k^2*N3 + ...
        M = np.zeros_like(matrices_N[0], dtype=float)
        for i in range(r):
            M += (k**i) * matrices_N[i]

        # Find eigenvalues and check if they are distinct
        eigenvalues = np.linalg.eigvals(M)

        # Check for distinctness within a small numerical tolerance
        unique_eigenvalues = True
        for i in range(len(eigenvalues)):
            for j in range(i + 1, len(eigenvalues)):
                if np.isclose(eigenvalues[i], eigenvalues[j]):
                    unique_eigenvalues = False
                    break
            if not unique_eigenvalues:
                break

        if unique_eigenvalues:
            # Found a good k, proceed with this M
            break
    else:
        # This part should theoretically never be reached
        raise RuntimeError("Could not find a deterministic combination with distinct eigenvalues.")
    # Find eigenvectors of M
    _, eigenvectors = np.linalg.eig(M)

    # Normalise eigenvectors to get characters
    char_table = []
    for i in range(r):
        v = eigenvectors[:, i]

        # Derive the unscaled character
        psi = np.array([v[j] / class_sizes[j] for j in range(r)])

        # Calculate the dimension squared
        psi_e = psi[0]
        norm_sum = np.sum([size * abs(val)**2 for size, val in zip(class_sizes, psi)])

        # This must be the square of a positive integer
        dim_squared = (order * abs(psi_e)**2) / norm_sum
        dim = int(round(np.sqrt(dim_squared).real))

        # Compute the scaling factor
        scaling_factor = dim / psi_e
        chi = psi * scaling_factor

        char_table.append(chi)

    # Sort table by dimension
    char_table.sort(key=lambda c: c[0].real)

    return np.array(char_table), classes, order

def print_table(table, classes):
    '''
    Formats and prints the character table.
    '''
    def pretty_complex(c, tol=1e-7):
        if abs(c.imag) < tol:
            return f"{c.real: .3f}".rstrip('0').rstrip('.')
        if abs(c.real) < tol:
            return f"{c.imag: .3f}j".rstrip('0').rstrip('.')
        return f"({c.real:.2f}{c.imag:+.2f}j)"

    header_sizes = "Size   | " + " | ".join(f"{len(c):^14}" for c in classes)
    print(header_sizes)
    print("-" * len(header_sizes))

    for char in table:
        dim = round(char[0].real)
        row_str = f"Dim {dim:<2} | " + " | ".join(f"{pretty_complex(v):^14}" for v in char)
        print(row_str)

In [3]:
GROUPS = {
    "S3": {"n": 3, "gens": ["(1,2)", "(1,2,3)"]},
    "A4": {"n": 4, "gens": ["(1,2,3)", "(2,3,4)"]},
    "S4": {"n": 4, "gens": ["(1,2)", "(1,2,3,4)"]},
    "P1": {"n": 8, "gens": ["(1,2,3,4)(5,6,7,8)", "(1,5,3,7)(2,8,4,6)"]}, # Q8
    "P2": {"n": 8, "gens": ["(1,2,3,4)(5,6,7,8)", "(1,8)(2,7)(3,6)(4,5)"]}, # D8
    "G1": {"n": 13,"gens": ["(1,2,3)(4,5,6)(7,8,9)(10,11,12)", "(2,9)(4,11)(5,8)(7,13)"]}, # PSL(2, 13)
    "G2": {"n": 14,"gens": ["(1,2,3,4)(5,6,7,8)(9,10,11,12)(13,14)", "(1,3,14,5,11,7)(2,4,6,10,8,13)"]}, # PSL(2, 17)
    "G3": {"n": 8, "gens": ["(1,2,3,4,5,6,7)", "(1,4)(2,3)(5,8)(6,7)"]}, # C7 ⋊ C8
    #"G4": {"n": 10,"gens": ["(1,2,3,4)(5,6,7,8)", "(3,9,8,10)(4,6,11,7)"]} # M11
}

group_results = {}

for name, data in GROUPS.items():
    print(f"\nCharacter Table for {name}")
    n = data["n"]
    gens = [parse_cycle_notation(s, n) for s in data["gens"]]

    table, classes, order = compute_character_table(gens)
    group_results[name] = {'table': table, 'classes': classes, 'order': order}

    print(f"Group order: {order}")
    print(f"Number of conjugacy classes: {len(classes)}")
    print_table(table, classes)

# An isomorphism implies identical character tables (up to ordering).
# Our canonical sorting should make them identical if isomorphic.
t1 = group_results["P1"]["table"]
t2 = group_results["P2"]["table"]
c1 = group_results["P1"]["classes"]
c2 = group_results["P2"]["classes"]

class_sizes1 = sorted([len(c) for c in c1])
class_sizes2 = sorted([len(c) for c in c2])

print(f"P1 Order: {group_results['P1']['order']}, Class sizes: {class_sizes1}")
print(f"P2 Order: {group_results['P2']['order']}, Class sizes: {class_sizes2}")

if np.allclose(t1, t2):
    print("\nThe character tables are identical.")
else:
    print("\nThe character tables are different.")
# Groups with the same character table may not be isomorphic.
# P1 is Q8, P2 is D8 which have the same character table but are not isomorphic.


Character Table for S3
Group order: 6
Number of conjugacy classes: 3
Size   |       1        |       2        |       3       
---------------------------------------------------------
Dim 1  |        1       |        1       |        1      
Dim 1  |        1       |        1       |       -1      
Dim 2  |        2       |       -1       |        0      

Character Table for A4
Group order: 12
Number of conjugacy classes: 4
Size   |       1        |       3        |       4        |       4       
--------------------------------------------------------------------------
Dim 1  |        1       |        1       |        1       |        1      
Dim 1  |        1       |        1       | (-0.50-0.87j)  | (-0.50+0.87j) 
Dim 1  |        1       |        1       | (-0.50+0.87j)  | (-0.50-0.87j) 
Dim 3  |        3       |       -1       |        0       |        0      

Character Table for S4
Group order: 24
Number of conjugacy classes: 5
Size   |       1        |       3        |      